In [1]:
load('../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [3]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [4]:
Poly_Ring.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i_rad3*c})

CPU times: user 3.93 s, sys: 3.89 ms, total: 3.93 s
Wall time: 3.93 s


In [6]:
# Find simmetries

In [7]:
%time adm_SE9 = SE9.find_admissible_projectivities()
len(adm_SE9)

CPU times: user 4.81 s, sys: 169 ms, total: 4.98 s
Wall time: 54.3 s


1296

In [8]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)
len(simm_SE9)

CPU times: user 2.59 s, sys: 137 ms, total: 2.72 s
Wall time: 7.44 s


54

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_perms = [Permutation(SE9.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE9]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(18, '(C3 x C3) : C2')

In [11]:
### geometric interpretation of the group of symmetries

In [12]:
load("../Imports/GeneralMethods.sage")
## Action of the stabilizer of SE4 on the 27 lines:
Ga = perm_group_lines(SE9, simm_SE9)

## Action of the stabilizer of SE4 on the 45 tritangent planes:
Gb = perm_group_planes(SE9, simm_SE9)

print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
((C3 x C3) : C3) : C2

Structure of the group Gb:
((C3 x C3) : C3) : C2


In [13]:
## The nine points are coplanar:

In [14]:
matrix(SE9.eckardt_points).rank()

3

In [15]:
matrix(SE9.eckardt_points[:3]).rank()

3

In [16]:
## plane which contains all the Eckardt points:
det(matrix([[x, y, z, t]]+SE9.eckardt_points[:3]))

(i_rad3 - 1)*x + y - z

In [17]:
print("Allignments of the nine Eckardt points:")
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            M = matrix([SE9.eckardt_points[i],SE9.eckardt_points[j],SE9.eckardt_points[k]])
            if M.rank() < 3:
                print((i, j, k))

Allignments of the nine Eckardt points:
(0, 1, 7)
(0, 2, 8)
(0, 3, 5)
(0, 4, 6)
(1, 2, 4)
(1, 3, 8)
(1, 5, 6)
(2, 3, 6)
(2, 5, 7)
(3, 4, 7)
(4, 5, 8)
(6, 7, 8)


In [18]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [19]:
OGb

[[1, 10, 11, 18, 24, 30, 38, 40, 44],
 [2, 27, 32, 8, 15, 21, 19, 45, 37],
 [3, 22, 33, 7, 20, 42, 34, 14, 26],
 [4, 17, 6, 25, 39, 35, 13, 28, 43],
 [5, 12, 31, 9, 36, 16, 23, 29, 41]]

In [20]:
## Decomposition of each element of Ga into disjoint cycles
## (expressed as simbolic names of lines)

In [21]:
for ii in range(54):
    gg = Ga[ii]
    print(str(ii)+':')
    cycles_gg = gg.cycles()
    print(cycles_gg)
    ## for cc in cycles_gg:
    ##    print([simbolic_line_from_index(i) for i in cc.cycle_tuples()[0]])
    ## print('')

0:
[(1,6,15,17,10,7), (2,16,9,21,18,22), (3,13,11,24,23,25), (4,12,26), (5,14,8,27,20,19)]
1:
[(1,15,10), (2,9,18), (3,11,23), (4,26,12), (5,8,20), (6,17,7), (13,24,25), (14,27,19), (16,21,22)]
2:
[(1,10,15), (2,18,9), (3,23,11), (4,12,26), (5,20,8), (6,7,17), (13,25,24), (14,19,27), (16,22,21)]
3:
[(1,5,15,8,10,20), (2,17,9,7,18,6), (3,26,11,12,23,4), (13,25,24), (14,16,27,21,19,22)]
4:
[(1,22,23), (2,6,25), (3,15,16), (4,19,8), (5,12,27), (7,24,18), (9,17,13), (10,21,11), (14,20,26)]
5:
[(1,20,10,8,15,5), (2,6,18,7,9,17), (3,4,23,12,11,26), (13,24,25), (14,22,19,21,27,16)]
6:
[(1,7,10,17,15,6), (2,22,18,21,9,16), (3,25,23,24,11,13), (4,26,12), (5,19,20,27,8,14)]
7:
[]
8:
[(1,5,24), (2,22,26), (3,14,7), (4,18,21), (6,11,27), (8,25,15), (9,16,12), (10,20,13), (17,23,19)]
9:
[(1,2,27), (3,25,26), (4,23,24), (5,16,7), (6,8,21), (9,19,15), (10,18,14), (11,13,12), (17,20,22)]
10:
[(1,8,13), (2,16,4), (3,27,17), (5,25,10), (6,23,14), (7,11,19), (9,21,26), (12,18,22), (15,20,24)]
11:
[(1,8),

In [22]:
## definition of 4 good elements  chosen in a suitable way:
GG = PermutationGroup(Ga)
g1 = '(1,15,10)(2,9,18)(3,11,23)(4,26,12)(5,8,20)(6,17,7)(13,24,25)(14,27,19)(16,21,22)'
g2 = '(1,9,14)(2,19,10)(3,13,4)(5,21,17)(6,20,16)(7,8,22)(11,24,26)(12,23,25)(15,18,27)'
g3 = '(1,8,13)(2,16,4)(3,27,17)(5,25,10)(6,23,14)(7,11,19)(9,21,26)(12,18,22)(15,20,24)'
g4 = '(1,9)(2,10)(3,7)(4,8)(5,12)(6,11)(13,22)(15,18)(16,24)(17,23)(20,26)(21,25)'

##The elements g1, g2, g3, g4 generate the whole group of 54 elements of the symmetries of SE9:
GG == PermutationGroup([g1, g2, g3, g4])

True

In [23]:
## Construction of a normal subgroup of GG:
H = PermutationGroup([g1, g2, g3])
H.is_normal(GG)

True

In [24]:
### structure of H:
H.structure_description()

'(C3 x C3) : C3'

In [25]:
## Construction of a normal subgroup of H
K = PermutationGroup([g1, g2])
K.is_normal(H)

True

In [26]:
## structure of K
K.structure_description()

'C3 x C3'

In [27]:
for h in [g1, g2, g3, g4]:
    print(perm_to_labels_lines(GG(h)))
    print()

[('E1', 'F14', 'G4'), ('E2', 'G3', 'F23'), ('E3', 'G5', 'F35'), ('E4', 'F46', 'G6'), ('E5', 'G2', 'F25'), ('E6', 'F16', 'G1'), ('F12', 'F36', 'F45'), ('F13', 'F56', 'F24'), ('F15', 'F26', 'F34')]

[('E1', 'G3', 'F13'), ('E2', 'F24', 'G4'), ('E3', 'F12', 'E4'), ('E5', 'F26', 'F16'), ('E6', 'F25', 'F15'), ('G1', 'G2', 'F34'), ('G5', 'F36', 'F46'), ('G6', 'F35', 'F45'), ('F14', 'F23', 'F56')]

[('E1', 'G2', 'F12'), ('E2', 'F15', 'E4'), ('E3', 'F56', 'F16'), ('E5', 'F45', 'G4'), ('E6', 'F35', 'F13'), ('G1', 'G5', 'F24'), ('G3', 'F26', 'F46'), ('G6', 'F23', 'F34'), ('F14', 'F25', 'F36')]

[('E1', 'G3'), ('E2', 'G4'), ('E3', 'G1'), ('E4', 'G2'), ('E5', 'G6'), ('E6', 'G5'), ('F12', 'F34'), ('F14', 'F23'), ('F15', 'F36'), ('F16', 'F35'), ('F25', 'F46'), ('F26', 'F45')]

